In [1]:
!pip install  openai langchain chromadb unstructured -q


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [8]:
from langchain.document_loaders.csv_loader import CSVLoader

def load_csvs(directory):
    loader = CSVLoader(directory)
    csv_data = loader.load()
    return csv_data

csv_data = load_csvs('data/Investors_v2.csv')
len(csv_data)


599

In [9]:
print(csv_data[0])

page_content='\ufeffInvestor Name: Frank M. Creer\nFund from Fund Master: Zone Ventures\nIndustry: Commercial Services,Software\nIndustry Copy: Software and Information Services\nVertical: Industrials,TMT\nInvestor Type: Venture Capital\nWebsite: www.zonevc.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 2.50\nMin check size: 1.00\nHQ Location: Menlo Park, CA\nURL (from Fund from Fund Master): www.zonevc.com\nPreferred TAM: Less than a Billion\nImpact Investor: No\n% Diversity in Managers/LPs: 25%\nAge of Fund/Investor Since (Years): 20.0\nNumber of investments (Portfolio Size): 20\nReturn Expectations: 10X and above\nExit Strategy Preference: IPO\nNetwork and Support Resources: Heavily Supportive\nDue diligence process: Heavily Involved\nDecision-making timeline: Months\nReputation: 4\nPortfolio Success Track Record (

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(csv_data)
print(len(docs))


921


In [7]:
docs[0]

Document(page_content='\ufeffInvestor Name: Frank M. Creer\nFund from Fund Master: Zone Ventures\nIndustry: Commercial Services,Software\nIndustry Copy: Software and Information Services\nVertical: Industrials,TMT\nInvestor Type: Venture Capital\nWebsite: www.zonevc.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 2.50\nMin check size: 1.00\nHQ Location: Menlo Park, CA\nURL (from Fund from Fund Master): www.zonevc.com\nPreferred TAM: Less than a Billion\nImpact Investor: No\n% Diversity in Managers/LPs: 25%\nAge of Fund/Investor Since (Years): 20.0\nNumber of investments (Portfolio Size): 20\nReturn Expectations: 10X and above\nExit Strategy Preference: IPO\nNetwork and Support Resources: Heavily Supportive\nDue diligence process: Heavily Involved\nDecision-making timeline: Months\nReputation: 4\nPortfolio Success Track

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/Volumes/T7/ML/minicoda3/envs/hugfac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [15]:
query = "Name different fund masters that normally invest in healthcare?"
matching_docs = db.similarity_search(query)

matching_docs[1]


Document(page_content='\ufeffInvestor Name: Michael Y. Mashaal\nFund from Fund Master: HealthCor Partners Management\nIndustry: Healthcare Devices and Supplies,Healthcare Services,Pharmaceuticals and Biotechnology\nIndustry Copy: \nVertical: Digital Health,HealthTech,Life Sciences,Manufacturing\nInvestor Type: Growth/Expansion\nWebsite: www.healthcorpartners.com\nPreferred Geography (from Fund from Fund Master): United States\nPreferred Investment Stage: Early Stage,Late Stage,PE Growth/Expansion,Seed\nPreferred Investment Type: Convertible Debt,SAFE,Debt\nMax Check size: 40.00\nMin check size: 10.00\nHQ Location: New York, NY\nURL (from Fund from Fund Master): www.healthcorpartners.com\nPreferred TAM: \nImpact Investor: \n% Diversity in Managers/LPs: \nAge of Fund/Investor Since (Years): \nNumber of investments (Portfolio Size): \nReturn Expectations: \nExit Strategy Preference: \nNetwork and Support Resources: \nDue diligence process: \nDecision-making timeline: \nReputation: \nPortf

In [16]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()


In [21]:
import os
# Read API key from env file
api_key = os.getenv("OPENAI_KEY")


from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage


model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name, temperature=0)


In [23]:
messages = [
    SystemMessage(
        content="You are a Venture capatilist consultant who has 7+ years of experience in funding and investment domain."
    ),
    HumanMessage(
        content="Recommend at most 5 Fund Masters(from the investors data you have) for each founder info that is provided to you in python dictionary format"
    ),
]
llm(messages)

AIMessage(content='Sure, please provide me with the founder info in Python dictionary format, and I will recommend at most 5 Fund Masters for each founder.', additional_kwargs={}, example=False)

In [24]:
template = (
    """You are a Venture capatilist consultant who has 7+ years of experience in funding and investment domain. You look into founders information and recommend investors name with respect to the data I have provided you using langchian csvreader.
    You receive a founder_info in python dictionary format and return recommded investors"""
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [30]:
import csv

founders_data = []
with open('data/founder_v2.csv', 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)
    for row in reader:
        founders_data.append(dict(zip(headers, row)))

document = founders_data[1]


In [31]:
document

{'\ufeffName': 'Minara El-Rahman',
 'Company': 'Mora Cosmetics',
 'Email': 'minara@moraglam.com',
 'Industry': 'CPG',
 'Verticals': 'Beauty & Wellness,Marketplace',
 'Target Customer': 'B2C,B2B',
 'Geography': 'United States of America (USA)',
 'Target Market (Locations)': 'Canada,USA',
 'Investment Stage': 'SEED',
 'Planned Raise Amount': '$500K - $750K',
 'Market size': '$5Billion to $10Billion',
 'Business Revenue Model': 'Retail',
 'Stage of development': 'Growth Phase',
 'Revenue/profitability': 'Pre-Revenue',
 'Fundraising history': 'Bootstrapped',
 'Traction': 'Unpaid customers,Unpaid Pilot',
 'Team Key Roles & experience': 'Strong',
 'Intellectual property': 'Strong ',
 'Competitive landscape': 'No Competition',
 'Customer Base (# paying clients)': '2.0',
 'Growth potential': 'Medium',
 'Management experience': 'Domain Expert,First time founder'}

In [34]:
from langchain.chains import RetrievalQA
query = """
TASK: Recommend investors for a founder by matching features that a investor would consider while investing in a startup? \n
INSTRUCTION: While matching investors with founders, please keep in mind that investors always invent in startups that match
thier industry and verticals. eg: A founder who is developing a heathcare product, usually attract investors that invest 
in healthcare base industry and verticals. Please give importance to features like geography, TAM and other relevant features.
Please restrict yourself from recommending investors that don't invest in domain and vertical of a founder. \n
QUESTION: Recommend at most 5 Fund Masters(from the investors data you have) for {0}. While recommending please keep in mind that that investor should be investing in that vertical and Industry""".format(document)
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
out=retrieval_chain.run(query)


In [35]:
out.split("\n")

['Based on the given information, here are five Fund Masters that may be a good fit for Mora Cosmetics:',
 '',
 '1. Investor Name: Austin Walne',
 '   Fund from Fund Master: ARTIS Ventures',
 '   Industry: Healthcare, Software',
 '   Vertical: Beauty & Wellness, Marketplace',
 '   Preferred Geography: United States',
 '   Preferred Investment Stage: Early Stage, Seed',
 '   Preferred Investment Type: Convertible Debt, SAFE, Debt',
 '   Max Check size: $15.00',
 '   Min check size: $1.00',
 '   HQ Location: San Francisco, CA',
 '   Website: www.av.co',
 '',
 '2. Investor Name: Simin Zhou',
 '   Fund from Fund Master: UL Ventures',
 '   Industry: Other Information Technology, Software',
 '   Vertical: Beauty & Wellness, Marketplace',
 '   Preferred Investment Stage: Early Stage, Seed',
 '   Preferred Investment Type: Convertible Debt, SAFE, Debt',
 '   Max Check size: $5.00',
 '   Min check size: $0.25',
 '   HQ Location: Northbrook, IL',
 '',
 '3. Investor Name: Bob Rees',
 '   Fund fro

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = """
ROLE: You are a Venture capatilist consultant who has 7+ years of experience in funding and investment domain. \n
TASK: Recommend investors for a founder by matching features that a investor would consider while investing in a startup? \n
INSTRUCTION: While matching investors with founders, please keep in mind that investors always invent in startups that match
thier industry and verticals. eg: A founder who is developing a heathcare product, usually attract investors that invest 
in healthcare base industry and verticals. Please give importance to features like geography, TAM and other relevant features.
Please restrict yourself from recommending investors that don't invest in domain and vertical of a founder. \n
QUESTION: Recommend at most 5 Fund Masters(from the investors data you have) for Sofia Yen who is the founder of PandiaHealth. 
        example for sophia yen, Shervin Pishevar(Fund maanger) would be a good option as he invests in Industires that work on healthcare."""
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer
